In [1]:
import numpy as np
from matplotlib import pyplot, cm
%matplotlib inline

from tqdm import tqdm
import time

import sys
np.set_printoptions(threshold=sys.maxsize)

# Set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [2]:
g0=4     #Bacterial growth Parameter
gamma=16     #Bacterial growth Parameter
k_value=42.5     #Bacterial growth Parameter

Dc=1e-3
Db=.21

t_ref=20
Xe_value=.3

X0=1e-3

C0=15
C_thresh_value=1.5

d=20 #separation between inoculum change; check for 5mm, 10mm, 20mm
rho=2.5
mu=40

dx=dy = 0.25

print("Current time is: ", end="")
print(time.asctime( time.localtime(time.time())))


Current time is: Mon Nov  4 14:25:52 2019


In [3]:
X1=np.zeros((369,369),dtype=np.float64)   # 2-D array 
X2=np.zeros((369,369),dtype=np.float64)
C=np.zeros((369,369),dtype=np.float64)
m=np.zeros((369,369))
Xe=np.zeros((369,369))
C_thresh=np.zeros((369,369))
k=np.zeros((369,369))

#(taking x,y for coordinate positions) :
x=np.linspace(-184,184,num=369,dtype=int)
y=np.linspace(-184,184,num=369,dtype=int)
 
############# Defining Initial Conditions #############
# for X1 and X2
for i in range(len(x)):
    for j in range(len(y)):
        if  ((x[i]-d/2)**2 + y[j]**2 <= rho**2):
            X1[i,j] = X0
        
        if  (x[i]+d/2)**2 + y[j]**2 <= rho**2:
            X2[i,j] = X0


                                                                       #np.savetxt('output.txt',X1,fmt='%.4e')

#Initializing parameters in array for all positional coordinates on 2D grid:
for i in range(len(x)):
    for j in range(len(y)):
        C[i,j]=C0
        C_thresh[i,j]=C_thresh_value
        Xe[i,j]=Xe_value
        k[i,j]=k_value



print("Initializing Initial Conditions has been completed")
#######################################################        
print("Current time is: ", end="")
print(time.asctime( time.localtime(time.time())))

Initializing Initial Conditions has been completed
Current time is: Mon Nov  4 14:25:59 2019


In [ ]:
def H_t(var,value):        # heavy side step function for time, not for matrix output
    if var-value <0:
        return 0
    if var-value==0:
        return 0.5
    else: return 1

    
def H_pos(mat1,mat2):      # heavy side step function for position, for matrix output H(Xe,X1+X2)
    
    mat3=np.zeros((max(mat1.shape),min(mat1.shape)))
    for i in range(max(mat1.shape)):
        for j in range(min(mat1.shape)):
            if mat1[i,j]-mat2[i,j]<0:
                mat3[i,j]=0
            if mat1[i,j]-mat2[i,j]==0:
                mat3[i,j]=0.5
            else: mat3[i,j]=1
  
    return mat3
       

def growth(nt,time_length):  # to be given number of time steps
    
    # nt : number of time-steps -- to be made 5001
    # time_length in hours: duration of time upto which you wish to see the simulation --- to be made 48 hours

    dt=time_length/nt       # time-step size
    print("The growth for time by time step %f has been completed for:-" %dt)
    
    for n in range(nt):
        
        g_c=g0*np.divide(C,np.add(k,C))
        
        m[:,:]=mu*np.multiply(np.multiply(X1[:,:],X2[:,:]),H_pos(Xe,X1+X2))*H_t((n-1)*dt,t_ref)
        X1[1:-1,1:-1]=X1[1:-1,1:-1]+dt*((np.multiply(g_c[1:-1,1:-1],X1[1:-1,1:-1]))+Db*np.multiply(H_pos(C[1:-1,1:-1],C_thresh[1:-1,1:-1]),(((X1[2:,1:-1]-2*X1[1:-1,1:-1]+X1[:-2,1:-1])/dx**2) + ((X1[1:-1,2:]-2*X1[1:-1,1:-1]+X1[1:-1,:-2])/dy**2))) - m[1:-1,1:-1])
        X2[1:-1,1:-1]=X2[1:-1,1:-1]+dt*((np.multiply(g_c[1:-1,1:-1],X2[1:-1,1:-1]))+Db*np.multiply(H_pos(C[1:-1,1:-1],C_thresh[1:-1,1:-1]),(((X2[2:,1:-1]-2*X2[1:-1,1:-1]+X2[:-2,1:-1])/dx**2) + ((X2[1:-1,2:]-2*X2[1:-1,1:-1]+X2[1:-1,:-2])/dy**2))) - m[1:-1,1:-1])
        
        C[1:-1,1:-1]= C[1:-1,1:-1] +dt*(-gamma*(np.multiply(g_c[1:-1,1:-1],np.add(X1[1:-1,1:-1],X2[1:-1,1:-1]))) + Dc*(((C[2:,1:-1]-2*C[1:-1,1:-1]+C[:-2,1:-1])/dx**2) + ((C[1:-1,2:]-2*C[1:-1,1:-1]+C[1:-1,:-2])/dy**2)))

        ############# Start: Boundary Conditions ######################
        
        #Neumann Condition: Normal component of Bacterial density on boundary is zero
        
        X1[:][-1] = X1[:][-2]
        X1[:][0]  = X1[:][1] 
        X1[-1][:] = X1[-2][:]
        X1[0][:]  = X1[1][:] 

        
                            # not sure whether it is a right way of doing. Perhaps, it should be opposite way as above calculation stencil never go on edge, so it would remain X0 eternally. But, it should change
        X2[:][-1] = X2[:][-2]
        X2[:][0]  = X2[:][1] 
        X2[-1][:] = X2[-2][:]
        X2[0][:]  = X2[1][:] 

        
        
        #Dirichlet condition: Fixed and Unchanging Concentration of growth media on boundary
        C[0][:]=C[-1][:]=C[:][0]=C[:][-1]=C0
        
        ############# End: Boundary Conditions ######################

        ############ Start: Saving history of Bacterial Density and Growth Media Concentration #########
        
        X1_hist[n+1][:][:]=X1[:][:]
        X2_hist[n+1][:][:]=X2[:][:]
        C_hist[n+1][:][:]=C[:][:]
        
        ############ End: Saving history of Bacterial Density and Growth Media Concentration #########
        
      
        print("%d steps, Wall clock time: "  %n, end="")
        
        print(time.asctime( time.localtime(time.time())))
        
nt=3000
time_length=28

X1_hist=np.zeros((nt+1,369,369))
X2_hist=np.zeros((nt+1,369,369))
C_hist=np.zeros((nt+1,369,369))

X1_hist[0][:][:]=X1[:][:]
X2_hist[0][:][:]=X2[:][:]
C_hist[0][:][:]=C[:][:]

growth(nt,time_length)

print("All the step calculations for growth have been completed")  

The growth for time by time step 0.009333 has been completed for:-
0 steps, Wall clock time: Mon Nov  4 14:26:02 2019
1 steps, Wall clock time: Mon Nov  4 14:26:05 2019
2 steps, Wall clock time: Mon Nov  4 14:26:07 2019
3 steps, Wall clock time: Mon Nov  4 14:26:09 2019
4 steps, Wall clock time: Mon Nov  4 14:26:12 2019
5 steps, Wall clock time: Mon Nov  4 14:26:15 2019
6 steps, Wall clock time: Mon Nov  4 14:26:18 2019
7 steps, Wall clock time: Mon Nov  4 14:26:20 2019
8 steps, Wall clock time: Mon Nov  4 14:26:21 2019
9 steps, Wall clock time: Mon Nov  4 14:26:23 2019
10 steps, Wall clock time: Mon Nov  4 14:26:25 2019
11 steps, Wall clock time: Mon Nov  4 14:26:27 2019
12 steps, Wall clock time: Mon Nov  4 14:26:30 2019
13 steps, Wall clock time: Mon Nov  4 14:26:31 2019
14 steps, Wall clock time: Mon Nov  4 14:26:33 2019
15 steps, Wall clock time: Mon Nov  4 14:26:36 2019
16 steps, Wall clock time: Mon Nov  4 14:26:38 2019
17 steps, Wall clock time: Mon Nov  4 14:26:40 2019
18 step

155 steps, Wall clock time: Mon Nov  4 14:31:46 2019
156 steps, Wall clock time: Mon Nov  4 14:31:48 2019
157 steps, Wall clock time: Mon Nov  4 14:31:51 2019
158 steps, Wall clock time: Mon Nov  4 14:31:53 2019
159 steps, Wall clock time: Mon Nov  4 14:31:55 2019
160 steps, Wall clock time: Mon Nov  4 14:31:57 2019
161 steps, Wall clock time: Mon Nov  4 14:32:00 2019
162 steps, Wall clock time: Mon Nov  4 14:32:02 2019
163 steps, Wall clock time: Mon Nov  4 14:32:04 2019
164 steps, Wall clock time: Mon Nov  4 14:32:07 2019
165 steps, Wall clock time: Mon Nov  4 14:32:09 2019
166 steps, Wall clock time: Mon Nov  4 14:32:11 2019
167 steps, Wall clock time: Mon Nov  4 14:32:13 2019
168 steps, Wall clock time: Mon Nov  4 14:32:16 2019
169 steps, Wall clock time: Mon Nov  4 14:32:18 2019
170 steps, Wall clock time: Mon Nov  4 14:32:20 2019
171 steps, Wall clock time: Mon Nov  4 14:32:22 2019
172 steps, Wall clock time: Mon Nov  4 14:32:24 2019
173 steps, Wall clock time: Mon Nov  4 14:32:2

309 steps, Wall clock time: Mon Nov  4 14:37:18 2019
310 steps, Wall clock time: Mon Nov  4 14:37:20 2019
311 steps, Wall clock time: Mon Nov  4 14:37:21 2019
312 steps, Wall clock time: Mon Nov  4 14:37:22 2019
313 steps, Wall clock time: Mon Nov  4 14:37:24 2019
314 steps, Wall clock time: Mon Nov  4 14:37:26 2019
315 steps, Wall clock time: Mon Nov  4 14:37:28 2019
316 steps, Wall clock time: Mon Nov  4 14:37:31 2019
317 steps, Wall clock time: Mon Nov  4 14:37:33 2019
318 steps, Wall clock time: Mon Nov  4 14:37:34 2019
319 steps, Wall clock time: Mon Nov  4 14:37:36 2019
320 steps, Wall clock time: Mon Nov  4 14:37:39 2019
321 steps, Wall clock time: Mon Nov  4 14:37:41 2019
322 steps, Wall clock time: Mon Nov  4 14:37:43 2019
323 steps, Wall clock time: Mon Nov  4 14:37:45 2019
324 steps, Wall clock time: Mon Nov  4 14:37:48 2019
325 steps, Wall clock time: Mon Nov  4 14:37:50 2019
326 steps, Wall clock time: Mon Nov  4 14:37:52 2019
327 steps, Wall clock time: Mon Nov  4 14:37:5

463 steps, Wall clock time: Mon Nov  4 15:07:54 2019
464 steps, Wall clock time: Mon Nov  4 15:07:55 2019
465 steps, Wall clock time: Mon Nov  4 15:07:56 2019
466 steps, Wall clock time: Mon Nov  4 15:07:57 2019
467 steps, Wall clock time: Mon Nov  4 15:07:58 2019
468 steps, Wall clock time: Mon Nov  4 15:07:59 2019
469 steps, Wall clock time: Mon Nov  4 15:08:00 2019
470 steps, Wall clock time: Mon Nov  4 15:08:02 2019
471 steps, Wall clock time: Mon Nov  4 15:08:03 2019
472 steps, Wall clock time: Mon Nov  4 15:08:04 2019
473 steps, Wall clock time: Mon Nov  4 15:08:05 2019
474 steps, Wall clock time: Mon Nov  4 15:08:06 2019
475 steps, Wall clock time: Mon Nov  4 15:08:07 2019
476 steps, Wall clock time: Mon Nov  4 15:08:08 2019
477 steps, Wall clock time: Mon Nov  4 15:08:09 2019
478 steps, Wall clock time: Mon Nov  4 15:08:10 2019
479 steps, Wall clock time: Mon Nov  4 15:08:11 2019
480 steps, Wall clock time: Mon Nov  4 15:08:12 2019
481 steps, Wall clock time: Mon Nov  4 15:08:1

617 steps, Wall clock time: Mon Nov  4 15:12:15 2019
618 steps, Wall clock time: Mon Nov  4 15:12:17 2019
619 steps, Wall clock time: Mon Nov  4 15:12:20 2019
620 steps, Wall clock time: Mon Nov  4 15:12:22 2019
621 steps, Wall clock time: Mon Nov  4 15:12:25 2019
622 steps, Wall clock time: Mon Nov  4 15:12:27 2019
623 steps, Wall clock time: Mon Nov  4 15:12:30 2019
624 steps, Wall clock time: Mon Nov  4 15:12:32 2019
625 steps, Wall clock time: Mon Nov  4 15:12:35 2019
626 steps, Wall clock time: Mon Nov  4 15:12:37 2019
627 steps, Wall clock time: Mon Nov  4 15:12:40 2019
628 steps, Wall clock time: Mon Nov  4 15:12:42 2019
629 steps, Wall clock time: Mon Nov  4 15:12:44 2019
630 steps, Wall clock time: Mon Nov  4 15:12:47 2019
631 steps, Wall clock time: Mon Nov  4 15:12:49 2019
632 steps, Wall clock time: Mon Nov  4 15:12:51 2019
633 steps, Wall clock time: Mon Nov  4 15:12:54 2019
634 steps, Wall clock time: Mon Nov  4 15:12:56 2019
635 steps, Wall clock time: Mon Nov  4 15:12:5

In [ ]:
print("Plotting figures for Nutrient Media Concentration")
for i in range(int(nt)):
    fig=pyplot.figure(figsize=(9.2,9.2))
    
    pyplot.xlabel('x')
    pyplot.ylabel('y')
    levels = np.linspace(0, 15, num=10) # the lowest point is being set zero because there is a negative bacterial density coming in picture otherwise
    contf = pyplot.contourf(x, y, C_hist[i][:][:], levels=levels)
    fig.suptitle('Concentration after %d time steps' %i)
    cbar = pyplot.colorbar(contf)
    cbar.set_label('C after %d time steps' %i)
    print(i)
    pyplot.axis('scaled', adjustable='box')
    ;

print("Current time is: ", end="")
print(time.asctime( time.localtime(time.time())))

#pyplot.figure(figsize=(9.2,9.2))
#pyplot.xlabel('x')
#pyplot.ylabel('y')
#levels = np.linspace(0, 100.0, num=51)
#contf = pyplot.contourf(x, y, X2, levels=levels)
#cbar = pyplot.colorbar(contf)
#cbar.set_label('X2 after %nt steps')
#pyplot.axis('scaled', adjustable='box');

In [ ]:
print("Plotting figures for X1 growth")
for i in range(int(nt)):
    fig=pyplot.figure(figsize=(9.2,9.2))

    pyplot.xlabel('x')
    pyplot.ylabel('y')
    levels = np.linspace(0, np.max(X1_hist[i]), num=30) # the lowest point is being set zero because there is a negative bacterial density coming in picture otherwise
    contf = pyplot.contourf(x, y, X1_hist[i], levels=levels)
    fig.suptitle('X1 after %d time steps' %i)
    cbar = pyplot.colorbar(contf)
    cbar.set_label('X1 after %d time steps' %i)
    print(i)
    pyplot.axis('scaled', adjustable='box')
    ;
print("Current time is: ", end="")
print(time.asctime( time.localtime(time.time())))


In [ ]:
print("Plotting figures for X2 growth")
for i in range(int(nt)):
    fig=pyplot.figure(figsize=(9.2,9.2))
    #pyplot.grid()
    pyplot.xlabel('x')
    pyplot.ylabel('y')
    levels = np.linspace(0, np.max(X2_hist[i]), num=30) # the lowest point is being set zero because there is a negative bacterial density coming in picture otherwise
    contf = pyplot.contourf(x, y, X2_hist[i], levels=levels)
    fig.suptitle('X2 after %d time steps' %i)
    cbar = pyplot.colorbar(contf)
    cbar.set_label('X2 after %d time steps' %i)
    print(i)
    pyplot.axis('scaled', adjustable='box')
    ;
print("Current time is: ", end="")
print(time.asctime( time.localtime(time.time())))

In [ ]:

############# Animation ################

from matplotlib import animation
from IPython.display import HTML

In [ ]:
def update_plot(nt, contf):

#    """
#     Update the line y-data of the Matplotlib figure.
#     
#     Parameters
#     ----------
#     n : integer
#         The time-step index.
#     u_hist : list of numpy.ndarray objects
#         The history of the numerical solution.
#     """
    fig.suptitle('Time step {:0>2}'.format(nt))
    contf.
# 
anim = animation.FuncAnimation(fig, update_plot,frames=nt, fargs=(contf,), interval=300)
# 							   
 	# Display the video.
HTML(anim.to_html5_video())

In [ ]:
import winsound
duration = 5000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)